In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def rotate(point, origin, angle_deg):
    angle_rad = np.deg2rad(angle_deg)
    return (point - origin) * np.exp(1j * angle_rad) + origin

In [ ]:
triangle_types = {1, 2} # type 1: for P2-tilings, type 2: for P3-tilings
shapes = {'A', 'O'} # 'A': acute equilateral triangle, 'O': obtuse equilateral triangle 
sides = {'F', 'B'} # 'F': frontside, 'B': backside
tolerance = 0.00000001

class RobinsonTriangle:
    def __init__(self, t_type, shape, side, A, B):
        if t_type not in triangle_types:
            raise ValueError('t_type must be in {}, but is {}'.format(triangle_types, t_type))
        self.t_type = t_type
        if shape not in shapes:
            raise ValueError('shape must be in {}, but is {}'.format(shapes, shape))
        self.shape = shape
        self.side = side
        if side not in sides:
            raise ValueError('side must be in {}, but is {}'.format(sides, side))
        self.side = side
        if abs(B - A) < tolerance:
            raise ValueError('points {} and {} too close'.format(A, B))
        if not isinstance(A, complex):
            raise ValueError('A must be a complex number but is {}'.format(type(A)))
        if not isinstance(B, complex):
            raise ValueError('B must be a complex number but is {}'.format(type(B)))
        self.A = A # the point at the top of the equilateral triangle as a complex number
        self.B = B # the point after A (anti-clockwise) as a complex number
        rot_angle_deg = 36 if self.shape == 'A' else 108
        self.C = rotate(B, A, rot_angle_deg)
    def __repr__(self):
        return '[t_type={}, shape={}, side={}, A={}, B={}, C={}]'.format(self.t_type, self.shape, self.side, self.A, self.B, self.C)
    def draw(self):
        plt.plot([self.A.real, self.B.real], [self.A.imag, self.B.imag], 
                 [self.A.real, self.C.real], [self.A.imag, self.C.imag], 
                 [self.B.real, self.C.real], [self.B.imag, self.C.imag],
                color='black'
                )
    def deflate(self):
        if self.t_type == 1:
            raise NotImplementedError('deflation for t_type 1 is not implemented yet')
        elif self.t_type == 2:
            if self.shape == 'A':
                if self.side == 'F':
                    D = rotate(self.C, self.B, 36)
                    return [
                        RobinsonTriangle(self.t_type, 'A', 'F', self.B, self.C),
                        RobinsonTriangle(self.t_type, 'O', 'B', D, self.A),
                    ] 
                elif self.side == 'B':
                    D = rotate(self.B, self.C, 360 - 36)
                    return [
                        RobinsonTriangle(self.t_type, 'O', 'F', D, self.C),
                        RobinsonTriangle(self.t_type, 'A', 'B', self.C, D),
                    ] 
            elif self.shape == 'O':
                if self.side == 'F':
                    P = rotate(self.A, self.B, 360 - 36)
                    Q = rotate(self.A, P, 36)
                    return [
                        RobinsonTriangle(self.t_type, 'O', 'B', Q, self.B),
                        RobinsonTriangle(self.t_type, 'A', 'F', P, self.A),
                        RobinsonTriangle(self.t_type, 'O', 'F', P, self.C),
                    ]
                elif self.side == 'B':
                    P = rotate(self.A, self.C, 36)
                    Q = rotate(self.A, P, 360 - 36)
                return [
                        RobinsonTriangle(self.t_type, 'O', 'F', Q, P),
                        RobinsonTriangle(self.t_type, 'A', 'B', P, Q),
                        RobinsonTriangle(self.t_type, 'O', 'B', P, self.A),
                    ]
            

In [ ]:
rt_2_A_F = RobinsonTriangle(2, 'A', 'F', complex(0), complex(1))
rt_2_A_F.draw()

In [ ]:
print(plt.figure().get_size_inches())
rt_2_A_F.draw()
for t in rt_2_A_F.deflate():
    t.draw()
print(plt.figure().get_size_inches())
print(2 * plt.figure().get_size_inches())
plt.figure(figsize= 2 * plt.figure().get_size_inches())
print(plt.figure().get_size_inches())
plt.show()

In [ ]:
rt_2_A_B = RobinsonTriangle(2, 'A', 'B', complex(0), complex(1))
rt_2_A_B.draw()

In [ ]:
rt_2_A_B.draw()
for t in rt_2_A_B.deflate():
    t.draw()
plt.show()

In [ ]:
rt_2_O_F = RobinsonTriangle(2, 'O', 'F', complex(0), complex(1))
rt_2_O_F.draw()

In [ ]:
rt_2_O_F.draw()
for t in rt_2_O_F.deflate():
    t.draw()
plt.show()

In [ ]:
rt_2_O_B = RobinsonTriangle(2, 'O', 'B', complex(0), complex(1))
rt_2_O_B.draw()

In [ ]:
rt_2_O_B.draw()
for t in rt_2_O_B.deflate():
    t.draw()
plt.show()

In [ ]:
def deflate_triangles(rts):
    res = []
    for rt in rts:
        res += rt.deflate()
    return res

In [ ]:
def deflate(first_rt, iter_depth):
    res = [first_rt]
    for i in range(iter_depth):
        res = deflate_triangles(res)
    return res

In [ ]:
fig_width = 50

In [ ]:
def get_box(rts):
    x_min = min([min(rt.A.real, rt.B.real, rt.C.real) for rt in rts])
    x_max = max([max(rt.A.real, rt.B.real, rt.C.real) for rt in rts])
    y_min = min([min(rt.A.imag, rt.B.imag, rt.C.imag) for rt in rts])
    y_max = max([max(rt.A.imag, rt.B.imag, rt.C.imag) for rt in rts])
    return (x_min, y_min), (x_max, y_max)

In [ ]:
def draw_rts(rts):
    (x_min, y_min), (x_max, y_max) = get_box(rts)
    y_scale = (x_max - x_min) / (y_max - y_min)
    plt.figure(figsize=(fig_width, fig_width / y_scale))
    plt.axis('equal')
    plt.axis('off')
    for t in rts:
        t.draw()
    plt.savefig("robinson_triangles.svg")
    plt.show()

In [ ]:
def draw_penrose_tiling_from_rts(rts):
    (x_min, y_min), (x_max, y_max) = get_box(rts)
    y_scale = (x_max - x_min) / (y_max - y_min)
    plt.figure(figsize=(fig_width, fig_width / y_scale))
    plt.axis('equal')
    plt.axis('off')
    n = len(rts)
    neigbouring_triangles = []
    for i in range(n):
        rt1 = rts[i]
        for j in range(i, n):
            rt2 = rts[j]
            if rt1.shape == rt2.shape and rt1.side != rt2.side:
                if abs(rt1.B - rt2.C) < tolerance and abs(rt1.C - rt2.B) < tolerance:
                    plt.plot(
                        [rt1.A.real, rt1.B.real], [rt1.A.imag, rt1.B.imag],
                        [rt1.A.real, rt1.C.real], [rt1.A.imag, rt1.C.imag],
                        [rt2.A.real, rt2.B.real], [rt2.A.imag, rt2.B.imag],
                        [rt2.A.real, rt2.C.real], [rt2.A.imag, rt2.C.imag],
                        color='black')

    plt.savefig("penrose_tiling.svg")
    plt.show()

In [ ]:
def draw_penrose_tiling(t_type, shape, side, iter_depth, A=complex(0), B=complex(1)):
    rt = RobinsonTriangle(t_type, shape, side, A, B)
    draw_penrose_tiling_from_rts(deflate(rt, iter_depth))

In [ ]:
draw_penrose_tiling(2, 'O', 'B', 8)

In [ ]:
draw_penrose_tiling(2, 'O', 'B', 8, A=rotate(complex(1), 0, 36), B=complex(0))

In [ ]:
res = deflate(rt_2_O_B, 8)

In [ ]:
draw_rts(res)

In [ ]:
draw_penrose_tiling_from_rts(res)